# Tarea3-Jose Maria Saldaño Guijarro

## Configuración

He creado un repositorio llamado  Tarea3-SM y lo he clonado en local con el comando "git clone <url>" .
Después he creado un entorno con el comando "conda create --name=tarea3" y lo activo con el comando "conda activate tarea3".
Instalo Python 3.10, ipykernel y JupyterLab en el entorno y añado el entorno conda al kernel de JupyterLab con el comando "python3 -m ipykernel install --user --name=tarea3".
Instalo JupyterLab en el entorno tarea3 y ejecuto JupyterLab y creo un notebook vacío seleccionando como kernel tarea3.



In [ ]:
# Importacion.
# import librosa
from scipy.io import wavfile
import IPython
import os
import numpy as np

## He tenido que cambiar el kernel porque tras instalar spicy seguía sin funcionar.

### Especificamos directorios de entrada y salida

In [ ]:
# Directorios que usaremos.
cwd = os.getcwd()
audio_input_path = os.path.join(cwd, os.path.join('audio', 'examples'))  
audio_output_path = os.path.join(cwd, os.path.join('audio', '_output'))
print(f'Directorio con los audios de entrada: {audio_input_path}')
print(f'Directorio donde guardaremos los audios generados: {audio_output_path}\n')

### Cargamos audio

In [ ]:
# Cargamos el archivo de audio.
filename = os.path.join(audio_input_path, 'breaking_bad.wav')
# audio_data, sample_rate = librosa.load(filename, sr=None, mono=False)
sample_rate, audio_data = wavfile.read(filename)
print(f'Frecuencia de muestreo (sample rate): {sample_rate/1000} kHz')

### Probamos el audio

In [ ]:
IPython.display.Audio(audio_data.T, rate=sample_rate)

### Mostramos características

In [ ]:
# Mostrar informacion del tamaño , canales, resolución (sonido estéreo).
print('Datos de audio (estereo):')
print(f'- Tamaño:     {audio_data.shape}')
print(f'- 1º canal:   {audio_data[:5, 0]}...')
print(f'- 2º canal:   {audio_data[:5, 1]}...')
print(f'- Resolucion: {type(audio_data[0,0])}\n')

### Convertimos de estéreo a mono y mostramos las características 

In [ ]:
# Convertimos a mono mediante la media por canal (simplificacion).
new_data_mono = audio_data.mean(axis=1)  # Column-wise.
print('Nuevos datos de audio (mono):')
print(f'- Nuevo tamaño: {new_data_mono.shape}')
print(f'- Canal unico:  {new_data_mono[:5]}...')

# Mantenemos la misma resolucion que antes.
new_data_mono = new_data_mono.astype(np.int16)
print(f'- Resolucion:   {type(new_data_mono[0])}\n')


### Probamos el audio mono

In [ ]:
IPython.display.Audio(new_data_mono, rate=sample_rate)


### Diferencia entre mono y estéreo

La gran diferencia es que en mono a la hora de grabar y reproducir sólo existe un único canal y en estéreo tenemos dos canales.